In [9]:
from model_builder_pme import model_builder
from data_gen import simulate_private_equity_cashflows
import numpy as np
from pme_calc import sp500_data, xirr, ln_pme, ks_pme, pme_plus, direct_alpha, moic, index_weighted_cashflows

In [10]:
cashflows = simulate_private_equity_cashflows()

In [11]:
# Normalize the cashflows of each fund in the market

def fund_weights(cashflows):
    funds = cashflows['FundID'].unique() 
    weights = {}
    for fund in funds:
        fund_cashflows = cashflows[cashflows['FundID'] == fund]
        weights[fund] = -fund_cashflows['Cashflow'].sum()
    return weights

cashflows_contr = cashflows[cashflows['Cashflow'] < 0]
eq_cashflows_contr = fund_weights(cashflows_contr)

def eq_calc(row):
    fund = row['FundID']
    cf = row['Cashflow']
    
    return cf / eq_cashflows_contr[fund]

# Apply the function row-wise
cashflows['eq_cashflow'] = cashflows.apply(eq_calc, axis=1)

# Drop and rename
cashflows = cashflows.drop(columns=['Cashflow']).rename(columns={'eq_cashflow': 'Cashflow'})

In [12]:
def portfolio_cashflows(cashflows, portfolio_size):
    # Randomly select fund IDs
    selected_funds = np.random.choice(cashflows['FundID'].unique(), size=portfolio_size, replace=False)
    
    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in selected_funds}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(selected_funds)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [13]:
port_cfs = portfolio_cashflows(cashflows, 200)
port_cfs = port_cfs.copy()
model = model_builder(cashflows, cashflows)
model

[*********************100%***********************]  1 of 1 completed


({'Portfolio KS-PME': 1.9028910842378766,
  'Market KS-PME': 1.9028910842378766,
  'Timing Alpha KS-PME': 2.220446049250313e-16,
  'Strategy Alpha KS-PME': 2.220446049250313e-16,
  'Geography Alpha KS-PME': 2.220446049250313e-16,
  'Sizing Alpha KS-PME': 0.0,
  'Residual Alpha KS-PME': -6.661338147750939e-16},
 {'Portfolio Direct Alpha': 0.03399607517871229,
  'Market Direct Alpha': 0.03399607517871229,
  'Timing Direct Alpha': -2.1510571102112408e-16,
  'Strategy Direct Alpha': -2.1510571102112408e-16,
  'Geography Direct Alpha': -2.1510571102112408e-16,
  'Sizing Direct Alpha': 0.0,
  'Residual Direct Alpha': 6.453171330633722e-16})

### Monte-Carlo simulation